LIGHT + PRO 1

В процессе перебора разных параметров были сделаны и light и pro-1. Не вижу смысла разделять их на два блока, так как оба задания подразумевают перебор разных параметров сети и обработки текста.
Было испробовано много параметров, но 0,65 val_dice_coef так и не был досстигнут. Однако средний процент распознования 85%  был достигнут уже на второй архитектуре, что соответствует цели задания pro-1. Но все равно попробовал другие варианты обработки, чтобы попробовать дадут ли они прирост по val_dice__coef.
ДУмаю, что причина того, что средний процент распознавания хороший, а val_dice_coef низки, кроется в том, что val_dice считает по фактически предсказанным значениям и сравнивает их с единицами в целевом наборе. А процент распознавания считает за единицу значения, превышающие порог ( в нашем случае 0,9). Это значит что много показателей в диапазоне от 0.9 до 1, но все-таки ближе к нижней границе.

Оставил только несколько архитектур:
- во-первых заменил самописный Conv1DTranspose на керасовский ( в текущих версиях он вссе-таки есть, вдруг даст прирост)
- далее убрал батчнормализацию, так как переобучения по текущей модели не наблюдаю
- добавил в UNET еще по слою свертки
- менял параметры обработки текста - увеличивал/снижал окно при формировании xTrain и yTrain, менял размер эмбеддинг-пространства и тд
- вместо WordToVec исспользовал свой слой Embedding, для чего формировал xTrain не по словам, а по индексам и их подавал в Эмбеддинг.

Выводы: с текущими архитектурами достичь 65 val_dice_coef не удаеттся на этой базе договоров. Очень низкий процент распознавания тэга s1. Скорее всего в этих тэгах содержатся самые общие слова, которые трудно однозначно относить к одному их тэгов.
Как вариант дальнейшей работы - попробовать усложнять сеть, например объединив UNET и PSPnet

# Подготовка данных

In [ ]:
!pip install pymorphy2 # установка лемматизатора pymorphy2

     |████████████████████████████████| 61kB 2.0MB/s 
     |████████████████████████████████| 8.2MB 7.3MB/s 


## Подключение библиотек

In [ ]:
import numpy as np # Библиотека для работы с массивами данных
from tensorflow.keras.models import Model, load_model # Импортируем модель, метод загрузки предобученной сети 
import re # Импортируем для работы со строками
from tensorflow.keras.preprocessing.text import Tokenizer # Методы для работы с текстами и преобразования их в последовательности
# Импортируем слои для сеток
from tensorflow.keras.layers import Dense, Embedding, Input, concatenate, Activation, MaxPooling1D, Conv1D, Conv1DTranspose, BatchNormalization, Dropout, Conv2DTranspose, Lambda
from tensorflow.keras import backend as K # Импортируем для подсчета dice_coef
from tensorflow.keras.optimizers import Adam, Adadelta, RMSprop # Импортируем оптимизаторы 
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import utils # Импортируем для работы с категориальными данными 
from google.colab import files # Импортируем для работы с файлами
import matplotlib.pyplot as plt # Импортируем для отрисовки графиков
from gensim.models import word2vec # Импортируем gensim 
import os # Импортируем для работы с файловой системой
import pandas as pd # И мпортируем для работы с DataFrame
import pymorphy2 # Импортируем для работы с леммами
import time # Импортируем для подсчета времени 
import seaborn as sns
sns.set_style('darkgrid')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Функции для работы с текстом

*Разбор данного раздела:* https://youtu.be/YePSAMoX49A?t=854

Преобразуем тесты договоров в списке слов

In [ ]:
# Чтение файла текста из файла, очитска от знаков препинания
def readText(fileName):
  f = open(fileName, 'r') #Открываем наш файл для чтения и считываем из него данные 
  text = f.read() #Записываем прочитанный текст в переменную 
  # Определяем символы для удаления
  delSymbols = ['\n', "\t", "\ufeff", ".", "_", "-", ",", "!", "?", "–", "(", ")", "«", "»", "№", ";"]

  for dS in delSymbols: # Каждый символ в списке символов для удаления
    text = text.replace(dS, " ") # Удаляем, заменяя на пробел

  # Ищем шаблоны и символы в строке и меняем на указанную подстроку
  text = re.sub("[.]", " ", text) 
  text = re.sub(":", " ", text)
  text = re.sub("<", " <", text)
  text = re.sub(">", "> ", text)

  # Метод split разделит текст по пробелам (а их может быть несколько после удаления символов)
  # При помощи метода join запишем все разделенные слова снова в строку
  text = ' '.join(text.split()) 

  text = text.lower() # Переводим текст в нижний регистр
  return text # Возвращаем тексты

# Преобразование исходного текста в список из слов в нормальной форме 
def text2Words(text):
  morph = pymorphy2.MorphAnalyzer() # Создаем экземпляр класса MorphAnalyzer
  words = text.split(' ') # Разделяем текст на пробелы
  words = [morph.parse(word)[0].normal_form for word in words] #Переводим каждое слово в нормалную форму  
  return words # Возвращаем слова


## Чтение данных 

*Разбор данного раздела:* https://youtu.be/YePSAMoX49A?t=1249

In [ ]:
directory = '/content/drive/MyDrive/27 Сегментация теста/Малая база/' # Путь к папке с договорами
os.listdir(directory)[250:255] # Выведем названия 5-ти договоров

['dogovor-zaloga-privatizirovannoy-kvartiri.txt',
 'dogovor-ustupki-prav-payschika.txt',
 'dogovor-zaloga-imuschestva-kvartiri.txt',
 'dogovor-zaloga-nedvizhimosti.txt',
 'dogovor-zaloga-sobstvennosti-na-kvartirui-v-stroyaschemsya-dome (1).txt']

In [ ]:
print('Всего', len(os.listdir(directory)), 'договора.')

Всего 432 договора.


In [ ]:
agreements = [] # Список, в который запишем все наши договоры
for filename in os.listdir(directory): # Проходим по всем файлам в директории договоров
  txt = readText(directory + filename) # Читаем текст договора
  if txt != '': # Если текст не пустой
    agreements.append(readText(directory + filename)) # Преобразуем файл в одну строку и добавляем в agreements

In [ ]:
n = 4
print(os.listdir(directory)[n]) # Выведем название n-ого договора
agreements[n] # Текст n-го договора

2 (4).txt


'договор бартера г г в лице действующего на основании именуемый в дальнейшем сторона 1 с одной стороны и в лице действующего на основании именуемый в дальнейшем сторона 2 с другой стороны именуемые в дальнейшем стороны заключили настоящий договор в дальнейшем договор о нижеследующем 1 предмет договора 1 1 <s1> сторона 1 принимает на себя обязательства поставить стороне 2 в количестве <s3> по цене на общую сумму рублей </s1> </s3> 1 2 <s1> сторона 2 принимает на себя обязательства принять и отгрузить по реквизитам стороны 1 в количестве <s3> по цене на общую сумму рублей </s1> </s3> 1 3 <s1> качество товара должно соответствовать </s1> 2 порядок поставки товара 2 1 <s1> товар передается в месте нахождения принимающей стороны </s1> с этого момента обязательство по поставке и передаче права собственности считается выполненным 2 2 <s3> доставка производится за счет </s3> 2 3 доставка производится 3 сроки поставки и передачи 3 1 <s1> <s4> сторона 1 обязуется поставить стороне 2 в срок до го

In [ ]:
words = [] # Здесь будут храниться все договора в виде списка слов
curTime = time.time() # Засечем текущее время
for i in range(len(agreements)): # Проходимся по всем договорам
  words.append(text2Words(agreements[i])) # Преобразуем очередной договор в список слов и добавляем в words
print('На преобразование ушло:', round(time.time() - curTime, 2), 'с.')

На преобразование ушло: 135.18 с.


In [ ]:
wordsToTest = words[-10:] # Возьмем 10 текстов для финальной проверки обученной нейронной сети 
words = words[:-10] # Для обученающей и проверочной выборок возьмем все тексты, кроме последних 10

In [ ]:
print('Колличество текстов в выборке для финальной проверки в конце ноутбука:', len(wordsToTest)) 
print('Колличество договоров, взятых для обучающей и тестовой выборки:',len(words)) 
print('Колличество слов в первом договоре:', len(words[10])) # Количество слов в первом договоре

Колличество текстов в выборке для финальной проверки в конце ноутбука: 10
Колличество договоров, взятых для обучающей и тестовой выборки: 412
Колличество слов в первом договоре: 503


In [ ]:
print('Пример исходного текста:')
print(agreements[4][:62], '\n')
print('Тот же текст, представленный в виде списка из слов в начальной форме:')
print(words[4][:10])

Пример исходного текста:
договор бартера г г в лице действующего на основании именуемый 

Тот же текст, представленный в виде списка из слов в начальной форме:
['договор', 'бартер', 'г', 'г', 'в', 'лицо', 'действовать', 'на', 'основание', 'именовать']


## Преобразование текста в индексы: формирование xTrain

*Разбор данного раздела:* https://youtu.be/YePSAMoX49A?t=1675

In [ ]:
# lower=True - приводим слова к нижнему регистру
# char_level=False - просим токенайзер учитывать слова, а не отдельные символы
tokenizer = Tokenizer(lower=True, filters = '', char_level=False, oov_token='unknown')

tokenizer.fit_on_texts(words) # "Скармливаем" наши тексты, т.е. даём в обработку методу, который соберет словарь частотности
clean_voc = {} 

for item in tokenizer.word_index.items(): #Преобразуем полученный список 
  clean_voc[item[0]] = item[1] # В словарь, меняя местами элементы полученного кортежа 

In [ ]:
print('Словарь всех слов по частоте:') 
print(clean_voc, '\n')
print('Длинна словаря:', len(clean_voc))

tag = '<s1>'
print('Индекс тега', tag, ':' ,clean_voc[tag])

Словарь всех слов по частоте:
{'в': 1, 'договор': 2, 'и': 3, '2': 4, '1': 5, 'по': 6, 'сторона': 7, 'настоящий': 8, 'с': 9, 'на': 10, '3': 11, '<s1>': 12, '</s1>': 13, 'не': 14, '4': 15, 'или': 16, 'за': 17, 'он': 18, 'случай': 19, 'о': 20, '5': 21, '•': 22, '<s3>': 23, '</s3>': 24, 'право': 25, 'работа': 26, 'срок': 27, '<s4>': 28, 'от': 29, '</s4>': 30, 'при': 31, 'они': 32, '6': 33, 'лицо': 34, 'имущество': 35, 'быть': 36, 'арендатор': 37, 'другой': 38, 'день': 39, 'действовать': 40, 'для': 41, 'условие': 42, '7': 43, 'если': 44, 'к': 45, 'п': 46, 'дальнейший': 47, 'который': 48, 'обязательство': 49, 'а': 50, 'страховой': 51, 'сумма': 52, 'порядок': 53, 'работник': 54, 'законодательство': 55, 'арендодатель': 56, 'действие': 57, 'указанный': 58, 'счёт': 59, 'свой': 60, 'размер': 61, 'соответствие': 62, 'именовать': 63, 'из': 64, 'мочь': 65, 'обязать': 66, 'страховщик': 67, 'основание': 68, 'исполнение': 69, 'один': 70, 'страхователь': 71, 'обязанность': 72, 'иной': 73, 'год': 74, 'вп

In [ ]:
# Преобразовываем текст в последовательность индексов согласно частотному словарю
tok_agreem = tokenizer.texts_to_sequences(words) # Обучающие тесты в индексы

print("Взглянем на фрагмент обучающего текста:")
print("В виде оригинального текста:              ", words[4][:20])
print("Он же в виде последовательности индексов: ", tok_agreem[4][:20], '\n')

Взглянем на фрагмент обучающего текста:
В виде оригинального текста:               ['договор', 'доверительный', 'управление', 'доля', 'в', 'уставный', 'капиталь', 'ооо', 'г', 'г', 'в', 'лицо', 'действовать', 'на', 'основание', 'именовать', 'в', 'дальнейший', 'учредитель', 'с']
Он же в виде последовательности индексов:  [3, 137, 128, 472, 2, 804, 1800, 1801, 78, 78, 2, 35, 41, 11, 69, 64, 2, 48, 220, 10] 



## Формирование yTrain

*Разбор данного раздела:* https://youtu.be/YePSAMoX49A?t=1879

Каждому индексу (слову) из договора необходимо поставить в соответсвие вектор из 6-ти элементов, соответствующих тегам категорий

In [ ]:
# Собираем список индексов и dummy encoded вектора
def get01XSamples(tok_agreem, tags_index):
  tags01 = [] # Список для тегов
  indexes = [] # Здесь будут лежать индексы
 
  for agreement in tok_agreem: # Проходимся по каждому договору-списку
    tag_place = [0, 0, 0, 0, 0, 0] # Создаем вектор [0,0,0,0,0,0]
    for ex in agreement: # Проходимся по каждому слову договора
        if ex in tags_index: # Смотрим, если индекс оказался нашим тегом
          place = np.argwhere(tags_index==ex) # Записываем под каким местом лежит этот тег в своем списке
          if len(place)!=0: # Проверяем, чтобы тег действительно был
            if place[0][0]<6: # Первые шесть тегов в списке - открывающие
              tag_place[place[0][0]] = 1    # Поэтому ставим 1
            else: 
              tag_place[place[0][0] - 6] = 0  # Остальные теги закрывающие, так что меняем на ноль
        else:          
          tags01.append(tag_place.copy()) # Расширяем наш список с каждой итерацией. Получаем в конце длинный список из всех тегов в одном 
          indexes.append(ex) # Докидываем индекс слова в список индексов

  return indexes, tags01

# Получение списка слов из индексов
def reverseIndex(clean_voc, x):
  reverse_word_map = dict(map(reversed, clean_voc.items())) # Берем пары значений всего словаря и размечаем наоборот, т.е. value:key
  words = [reverse_word_map.get(letter) for letter in x] # Вытаскиваем по каждому ключу в список
  return words # Возвращаем полученный текст

In [ ]:
# s1 Условия
# s2 Запреты
# s3 Стоимость, всё про цены и деньги
# s4 Всё про сроки
# s5 Неустойка
# s6 Всё про адреса и геолокации

tags_index = ['<s' + str(i) + '>' for i in range(1, 7)] # Получаем список открывающих тегов
closetags = ['</s' + str(i) + '>' for i in range(1, 7)] # Получаем список закрывающих тегов
tags_index.extend(closetags) # Объединяем все теги

tags_index = np.array([clean_voc[i] for i in tags_index]) # Получаем из словаря частотности индексы всех тегов
print('Индексы тегов:', tags_index)

Индексы тегов: [ 13 344  24  29 119 275  14 356  25  31 120 268]


In [ ]:
xData, yData = get01XSamples(tok_agreem,tags_index) # Распознаем теги и создаем список с ними, с индексами
decoded_text = reverseIndex(clean_voc, xData) # Для создания списков с embedding-ами сначала преобразуем список индексов обратно в слова

In [ ]:
print('Длина xData:', len(xData))
print('Длина yData:', len(yData))

print('Посмотрим на исходный набор слов: ', words[0][50:80])
print('Посмотрим на декодированный текст:', decoded_text[50:80])
print('Посмотрим на часть из xData:     ', xData[50:80])
print('Посмотрим на часть из yData:     ', yData[50:80])

Длина xData: 411870
Длина yData: 411870
Посмотрим на исходный набор слов:  ['1', 'предмет', 'договор', '1', '1', '<s1>', 'по', 'настоящий', 'договор', 'хранитель', 'обязываться', 'хранить', 'вещь', 'передать', 'он', 'поклажедатель', 'и', 'возвратить', 'этот', 'вещь', 'в', 'сохранность', '</s1>', '1', '2', 'поклажедатель', 'передавать', 'на', 'хранение', 'по']
Посмотрим на декодированный текст: ['1', 'предмет', 'договор', '1', '1', 'по', 'настоящий', 'договор', 'хранитель', 'обязываться', 'хранить', 'вещь', 'передать', 'он', 'поклажедатель', 'и', 'возвратить', 'этот', 'вещь', 'в', 'сохранность', '1', '2', 'поклажедатель', 'передавать', 'на', 'хранение', 'по', 'настоящий', 'договор']
Посмотрим на часть из xData:      [5, 108, 2, 5, 5, 6, 8, 2, 220, 115, 914, 315, 124, 18, 240, 3, 539, 242, 315, 1, 411, 5, 4, 240, 299, 10, 159, 6, 8, 2]
Посмотрим на часть из yData:      [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0],

## Функции для обработки выборок и распознавания

In [ ]:
# Формируем выборку из индексов
def getSetFromIndexes(wordIndexes, xLen, step): 
  xBatch = [] # Лист для фрагментов текста
  wordsLen = len(wordIndexes) # Определяем длинну текста
  index = 0 # Задаем стартовый индекс
  
  while (index + xLen <= wordsLen): # Пока сумма индекса с длинной фрагмента меньше или равна числу слов в выборке
    xBatch.append(wordIndexes[index:index+xLen]) # Добавляем X в лист фразментов текста
    index += step # Сдвигаемся на step

  return xBatch # Лист для фрагментов текста

In [ ]:
# Функция, выводящая точность распознавания каждой категории отдельно
def recognizeSet(tagI, pred, tags, length, value):
  total=0

  for j in range(6): # общее количество тегов
    correct=0
    for i in range(len(tagI)): # проходимся по каждому списку списка тегов
      for k in range(length): # проходимся по каждому тегу
        if tagI[i][k][j]==(pred[i][k][j]>value).astype(int): # если соответствующие индексы совпадают, значит сеть распознала верно
          correct+=1 
    print("Сеть распознала категорию '{}' на {}%".format(tags[j],round(100*correct/(len(tagI)*length), 2)))
    total += 100 * correct / (len(tagI)*length)
  print("Cредняя точность {}%".format(total/6))

In [ ]:
# Создаем выборки
def getSets(model, senI, tagI):
  xVector = [] # Здесь будет лежать embedding представление каждого из индексов
  tmp = [] # Временный список
  for text in senI: # Проходимся по каждому тексту-списку
    tmp=[]
    for word in text: # Проходимся по каждому слову в тексте-списке
      tmp.append(model[word]) 

    xVector.append(tmp)

  return np.array(xVector), np.array(tagI)

## Функция подсчета ошибки и callback

*Разбор данного раздела:* https://youtu.be/YePSAMoX49A?t=3293


In [ ]:
# Функция, которая смотрит на пересечение областей. Нужна для accuracy
def dice_coef(y_true, y_pred):
    # print('11')
    return (2. * K.sum(y_true * y_pred) + 1.) / (K.sum(y_true) + K.sum(y_pred) + 1.)

In [ ]:
model_checkpoint_callback = ModelCheckpoint(
    '/content/drive/MyDrive/27 Сегментация теста/savings', monitor='val_dice_coef', verbose=0, save_best_only=True,
    save_weights_only=True, mode='auto', save_freq='epoch')

# my UNET 1 (с керасовский Conv1DTranspose)

## Подготовка данных

In [ ]:
xLen = 256 # Длина окна
step = 30 # Шаг 
embeddingSize = 300 # Количество измерений для векторного пространства

In [ ]:
# Генерируем наборы с заданными параметрами окна
xTrain = getSetFromIndexes(decoded_text, xLen, step) # Последовательность из xLen слов
yTrain = getSetFromIndexes(yData, xLen, step) # Последовательность из xLen-тегов

In [ ]:
modelGENSIM = word2vec.Word2Vec(xTrain, size = embeddingSize, window = 10, min_count = 1, workers = 10, iter = 10)

In [ ]:
xTrainGENSIM, yTrainGENSIM = getSets(modelGENSIM, xTrain, yTrain)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


## Создание модели

In [ ]:
# UNET с Conv1DTranspose из Кераса, а не самописный
def create_unet(k = 1, num_classes = 6, input_shape= (30, 300)):

        
    img_input = Input(input_shape) 

    # Block 1
    x = Conv1D(64 * k , 3, padding='same')(img_input) 
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(64 * k , 3, padding='same')(x)  
    x = BatchNormalization()(x)     
    block_1_out = Activation('relu')(x) 

    x = MaxPooling1D()(block_1_out)

    # Block 2
    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)  

    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_2_out = Activation('relu')(x)

    x = MaxPooling1D()(block_2_out)

    # Block 3
    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)               
    x = Activation('relu')(x)                     

    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_3_out = Activation('relu')(x)

    # x = block_3_out
    x = MaxPooling1D()(block_3_out)

    # Block 4
    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)      
    block_4_out = Activation('relu')(x)
    x = block_4_out 

    # UP 2
    x = Conv1DTranspose(256, kernel_size=2, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 

    x = concatenate([x, block_3_out]) 
    x = Conv1D(256 * k , 3, padding='same')(x) 
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # UP 3
    x = Conv1DTranspose(128, kernel_size=2, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 

    x = concatenate([x, block_2_out])
    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # UP 4
    x = Conv1DTranspose(64, kernel_size=2, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = concatenate([x, block_1_out])
    x = Conv1D(64 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(64 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(num_classes, 3, activation='sigmoid', padding='same')(x)

    model = Model(img_input, x) 

    model = Model(img_input, x)
    model.compile(optimizer=Adam(0.0025), 
                  loss='categorical_crossentropy',
                  metrics=[dice_coef])
  
    return model

In [ ]:
model_b_UNET = create_unet(input_shape=(xLen, embeddingSize))
model_b_UNET.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 300)]   0                                            
__________________________________________________________________________________________________
conv1d_10 (Conv1D)              (None, 256, 64)      57664       input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_10 (BatchNo (None, 256, 64)      256         conv1d_10[0][0]                  
__________________________________________________________________________________________________
activation_10 (Activation)      (None, 256, 64)      0           batch_normalization_10[0][0]     
_______________________________________________________________________________________

## Обучение

In [ ]:
model_b_UNET.compile(optimizer=Adam(0.0007), 
                  loss='categorical_crossentropy',
                  metrics=[dice_coef])
history = model_b_UNET.fit(xTrainGENSIM, yTrainGENSIM, epochs=40, batch_size=64, validation_split = 0.2) 

Epoch 1/40
173/173 [==============================] - 8s 45ms/step - loss: 0.1950 - dice_coef: 0.7763 - val_loss: 2.3017 - val_dice_coef: 0.5839
Epoch 2/40
173/173 [==============================] - 7s 42ms/step - loss: 0.1958 - dice_coef: 0.7736 - val_loss: 2.3619 - val_dice_coef: 0.5820
Epoch 3/40
173/173 [==============================] - 7s 42ms/step - loss: 0.1946 - dice_coef: 0.7784 - val_loss: 2.3783 - val_dice_coef: 0.5760
Epoch 4/40
173/173 [==============================] - 7s 42ms/step - loss: 0.1946 - dice_coef: 0.7791 - val_loss: 2.5101 - val_dice_coef: 0.5832
Epoch 5/40
173/173 [==============================] - 7s 42ms/step - loss: 0.1946 - dice_coef: 0.7789 - val_loss: 2.4677 - val_dice_coef: 0.5896
Epoch 6/40
173/173 [==============================] - 7s 43ms/step - loss: 0.1946 - dice_coef: 0.7778 - val_loss: 2.3927 - val_dice_coef: 0.5860
Epoch 7/40
173/173 [==============================] - 7s 43ms/step - loss: 0.1947 - dice_coef: 0.7775 - val_loss: 2.4371 - val_dic

Прироста не получили

## Проверка среднего распознавания

In [ ]:
# xLen = 256 # Длина окна
# step = 30 # Шаг 
# embeddingSize = 300 # Количество измерений для векторного пространства

# Генерируем наборы с заданными параметрами окна
xTest = getSetFromIndexes(decoded_text, xLen, step) # Последовательность из xLen слов
yTest = getSetFromIndexes(yDataTest, xLen, step) # Последовательность из xLen-тегов

In [ ]:
# Передаем в word2vec списки списков слов для обучения
# size = embeddingSize - размер эмбеддинга
# window = 10 - расстояние между текущим и прогнозируемым словом в предложении
# min_count = 1 - игнорирование всех слов с частотой, меньше, чем 1
# workers = 10 - число потоков обучения эмбеддинга
# iter = 10 - число эпох обучения эмбеддинга

modelGENSIM = word2vec.Word2Vec(xTest, size = embeddingSize, window = 10, min_count = 1, workers = 10, iter = 10)

In [ ]:
xTestGENSIM, yTestGENSIM = getSets(modelGENSIM, xTest, yTest)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [ ]:
print('Размерность полученного xTrain:', xTestGENSIM.shape)
print('Размерность полученного yTrain:', yTestGENSIM.shape)

Размерность полученного xTrain: (414, 256, 300)
Размерность полученного yTrain: (414, 256, 6)


In [ ]:
tags = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6']
# s1 Условия
# s2 Запреты
# s3 Стоимость, всё про цены и деньги
# s4 Всё про сроки
# s5 Неустойка
# s6 Всё про адреса и геолокации

In [ ]:
pred = model_b_UNET2.predict(xTestGENSIM) # сделаем предсказание

In [ ]:
recognizeSet(yTestGENSIM, pred, tags, xLen, 0.9)

Сеть распознала категорию 'S1' на 54.76%
Сеть распознала категорию 'S2' на 98.35%
Сеть распознала категорию 'S3' на 86.23%
Сеть распознала категорию 'S4' на 88.37%
Сеть распознала категорию 'S5' на 82.75%
Сеть распознала категорию 'S6' на 99.91%
Cредняя точность 85.06101497933885%


# my UNET 2 (без батчнормализации)

## Создание модели

In [ ]:
# Убираем Батчнормализацию
def create_unet2(k = 1, num_classes = 6, input_shape= (30, 300)):

       
    img_input = Input(input_shape) 

    # Block 1
    x = Conv1D(64 * k , 3, padding='same')(img_input) 
    # x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(64 * k , 3, padding='same')(x)  
    # x = BatchNormalization()(x)     
    block_1_out = Activation('relu')(x) 

    x = MaxPooling1D()(block_1_out)

    # Block 2
    x = Conv1D(128 * k , 3, padding='same')(x)
    # x = BatchNormalization()(x)
    x = Activation('relu')(x)  

    x = Conv1D(128 * k , 3, padding='same')(x)
    # x = BatchNormalization()(x)
    block_2_out = Activation('relu')(x)

    x = MaxPooling1D()(block_2_out)

    # Block 3
    x = Conv1D(256 * k , 3, padding='same')(x)
    # x = BatchNormalization()(x)               
    x = Activation('relu')(x)                     

    x = Conv1D(256 * k , 3, padding='same')(x)
    # x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(256 * k , 3, padding='same')(x)
    # x = BatchNormalization()(x)
    block_3_out = Activation('relu')(x)

    # x = block_3_out
    x = MaxPooling1D()(block_3_out)

    # Block 4
    x = Conv1D(512 * k , 3, padding='same')(x)
    # x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(512 * k , 3, padding='same')(x)
    # x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(512 * k , 3, padding='same')(x)
    # x = BatchNormalization()(x)      
    block_4_out = Activation('relu')(x)
    x = block_4_out 

    # UP 2
    x = Conv1DTranspose(256, kernel_size=2, strides=2, padding='same')(x)
    # x = BatchNormalization()(x)
    x = Activation('relu')(x) 

    x = concatenate([x, block_3_out]) 
    x = Conv1D(256 * k , 3, padding='same')(x) 
    # x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(256 * k , 3, padding='same')(x)
    # x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # UP 3
    x = Conv1DTranspose(128, kernel_size=2, strides=2, padding='same')(x)
    # x = BatchNormalization()(x)
    x = Activation('relu')(x) 

    x = concatenate([x, block_2_out])
    x = Conv1D(128 * k , 3, padding='same')(x)
    # x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(128 * k , 3, padding='same')(x)
    # x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # UP 4
    x = Conv1DTranspose(64, kernel_size=2, strides=2, padding='same')(x)
    # x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = concatenate([x, block_1_out])
    x = Conv1D(64 * k , 3, padding='same')(x)
    # x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(64 * k , 3, padding='same')(x)
    # x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(num_classes, 3, activation='sigmoid', padding='same')(x)

    model = Model(img_input, x) 

    model = Model(img_input, x)
    model.compile(optimizer=Adam(0.001), 
                  loss='categorical_crossentropy',
                  metrics=[dice_coef])
  
    return model

In [ ]:
model_b_UNET2 = create_unet2(input_shape=(xLen, embeddingSize))
model_b_UNET2.summary()

Model: "functional_27"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 256, 300)]   0                                            
__________________________________________________________________________________________________
conv1d_112 (Conv1D)             (None, 256, 64)      57664       input_8[0][0]                    
__________________________________________________________________________________________________
activation_124 (Activation)     (None, 256, 64)      0           conv1d_112[0][0]                 
__________________________________________________________________________________________________
conv1d_113 (Conv1D)             (None, 256, 64)      12352       activation_124[0][0]             
______________________________________________________________________________________

## Обучение

In [ ]:
history = model_b_UNET2.fit(xTrainGENSIM, yTrainGENSIM, epochs=40, batch_size=64, validation_split = 0.2,
                            callbacks=[model_checkpoint_callback]) # Скармливаем модели изображения, разделенные по 3 классам

Epoch 1/40
173/173 [==============================] - 6s 36ms/step - loss: 0.7081 - dice_coef: 0.4713 - val_loss: 0.6950 - val_dice_coef: 0.4976
Epoch 2/40
173/173 [==============================] - 6s 34ms/step - loss: 0.5038 - dice_coef: 0.5508 - val_loss: 0.8376 - val_dice_coef: 0.5198
Epoch 3/40
173/173 [==============================] - 6s 34ms/step - loss: 0.3823 - dice_coef: 0.6134 - val_loss: 0.9467 - val_dice_coef: 0.5346
Epoch 4/40
173/173 [==============================] - 6s 34ms/step - loss: 0.3256 - dice_coef: 0.6460 - val_loss: 1.3139 - val_dice_coef: 0.5413
Epoch 5/40
173/173 [==============================] - 6s 34ms/step - loss: 0.2814 - dice_coef: 0.6715 - val_loss: 1.0049 - val_dice_coef: 0.5374
Epoch 6/40
173/173 [==============================] - 6s 34ms/step - loss: 0.2578 - dice_coef: 0.6883 - val_loss: 1.2603 - val_dice_coef: 0.5485
Epoch 7/40
173/173 [==============================] - 6s 34ms/step - loss: 0.2445 - dice_coef: 0.6978 - val_loss: 1.2917 - val_dic

In [ ]:
# Пробуем дообучить модель
history = model_b_UNET2.fit(xTrainGENSIM, yTrainGENSIM, epochs=40, batch_size=32, validation_split = 0.2) # Скармливаем модели изображения, разделенные по 3 классам

Epoch 1/40
345/345 [==============================] - 7s 21ms/step - loss: 0.2442 - dice_coef: 0.7193 - val_loss: 1.2660 - val_dice_coef: 0.5599
Epoch 2/40
345/345 [==============================] - 7s 21ms/step - loss: 0.2140 - dice_coef: 0.7456 - val_loss: 1.4120 - val_dice_coef: 0.5759
Epoch 3/40
345/345 [==============================] - 7s 20ms/step - loss: 0.2291 - dice_coef: 0.7344 - val_loss: 1.4105 - val_dice_coef: 0.5728
Epoch 4/40
345/345 [==============================] - 7s 21ms/step - loss: 0.2086 - dice_coef: 0.7524 - val_loss: 1.5315 - val_dice_coef: 0.5803
Epoch 5/40
345/345 [==============================] - 7s 21ms/step - loss: 0.2150 - dice_coef: 0.7470 - val_loss: 1.3334 - val_dice_coef: 0.5728
Epoch 6/40
345/345 [==============================] - 7s 21ms/step - loss: 0.2172 - dice_coef: 0.7455 - val_loss: 1.5014 - val_dice_coef: 0.5706
Epoch 7/40
345/345 [==============================] - 7s 21ms/step - loss: 0.2281 - dice_coef: 0.7372 - val_loss: 1.6290 - val_dic

In [ ]:
model_b_UNET2.load_weights('/content/drive/MyDrive/27 Сегментация теста/savings')

In [ ]:
model_b_UNET3 = create_unet2(input_shape=(xLen, embeddingSize))
model_b_UNET3.summary()

Model: "functional_23"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 256, 300)]   0                                            
__________________________________________________________________________________________________
conv1d_95 (Conv1D)              (None, 256, 64)      57664       input_7[0][0]                    
__________________________________________________________________________________________________
activation_105 (Activation)     (None, 256, 64)      0           conv1d_95[0][0]                  
__________________________________________________________________________________________________
conv1d_96 (Conv1D)              (None, 256, 64)      12352       activation_105[0][0]             
______________________________________________________________________________________

In [ ]:
history = model_b_UNET3.fit(xTrainGENSIM, yTrainGENSIM, epochs=40, batch_size=64, validation_split = 0.2,
                            callbacks=[model_checkpoint_callback]) 

Epoch 1/40
173/173 [==============================] - 6s 36ms/step - loss: 0.7205 - dice_coef: 0.4673 - val_loss: 0.7092 - val_dice_coef: 0.4939
Epoch 2/40
173/173 [==============================] - 6s 34ms/step - loss: 0.5336 - dice_coef: 0.5403 - val_loss: 0.8743 - val_dice_coef: 0.4792
Epoch 3/40
173/173 [==============================] - 6s 35ms/step - loss: 0.4200 - dice_coef: 0.5905 - val_loss: 1.0461 - val_dice_coef: 0.5307
Epoch 4/40
173/173 [==============================] - 6s 34ms/step - loss: 0.3377 - dice_coef: 0.6426 - val_loss: 1.0824 - val_dice_coef: 0.5360
Epoch 5/40
173/173 [==============================] - 6s 34ms/step - loss: 0.2955 - dice_coef: 0.6672 - val_loss: 1.0406 - val_dice_coef: 0.5515
Epoch 6/40
173/173 [==============================] - 6s 34ms/step - loss: 0.2584 - dice_coef: 0.6906 - val_loss: 1.2986 - val_dice_coef: 0.5431
Epoch 7/40
173/173 [==============================] - 6s 34ms/step - loss: 0.2515 - dice_coef: 0.6961 - val_loss: 1.3123 - val_dic

## Проверка среднего предсказания

In [ ]:
pred = model_b_UNET2.predict(xTestGENSIM) # сделаем предсказание
recognizeSet(yTestGENSIM, pred, tags, xLen, 0.9)

Сеть распознала категорию 'S1' на 64.74%
Сеть распознала категорию 'S2' на 98.33%
Сеть распознала категорию 'S3' на 85.86%
Сеть распознала категорию 'S4' на 88.51%
Сеть распознала категорию 'S5' на 93.1%
Сеть распознала категорию 'S6' на 99.91%
Cредняя точность 88.40671390013495%


Как видим целевая точноссть достигнута. Остальными архитектурами будем пробовать досстичь цели по val_dice_coef

# MY_UNET 3 (увеличил окно)

## Подготовка

In [ ]:
xLen = 312 # Длина окна
step = 15 # Шаг 
embeddingSize = 300 # Количество измерений для векторного пространства

In [ ]:
# Генерируем наборы с заданными параметрами окна
xTrain = getSetFromIndexes(decoded_text, xLen, step) # Последовательность из xLen слов
yTrain = getSetFromIndexes(yData, xLen, step) # Последовательность из xLen-тегов

In [ ]:
modelGENSIM = word2vec.Word2Vec(xTrain, size = embeddingSize, window = 10, min_count = 1, workers = 10, iter = 10)

In [ ]:
xTrainGENSIM, yTrainGENSIM = getSets(modelGENSIM, xTrain, yTrain)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


## Создание модели

In [ ]:
model_b_UNET2 = create_unet2(input_shape=(xLen, embeddingSize))
model_b_UNET2.summary()

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 312, 300)]   0                                            
__________________________________________________________________________________________________
conv1d_17 (Conv1D)              (None, 312, 64)      57664       input_2[0][0]                    
__________________________________________________________________________________________________
activation_19 (Activation)      (None, 312, 64)      0           conv1d_17[0][0]                  
__________________________________________________________________________________________________
conv1d_18 (Conv1D)              (None, 312, 64)      12352       activation_19[0][0]              
_______________________________________________________________________________________

## Обучение

In [ ]:
history = model_b_UNET2.fit(xTrainGENSIM, yTrainGENSIM, epochs=40, batch_size=64, validation_split = 0.2,
                            callbacks=[model_checkpoint_callback]) # Скармливаем модели изображения, разделенные по 3 классам

Epoch 1/40
345/345 [==============================] - 14s 42ms/step - loss: 0.5811 - dice_coef: 0.5255 - val_loss: 0.7477 - val_dice_coef: 0.5278
Epoch 2/40
345/345 [==============================] - 14s 40ms/step - loss: 0.3568 - dice_coef: 0.6396 - val_loss: 1.0618 - val_dice_coef: 0.5555
Epoch 3/40
345/345 [==============================] - 14s 40ms/step - loss: 0.2680 - dice_coef: 0.6926 - val_loss: 1.1511 - val_dice_coef: 0.5600
Epoch 4/40
345/345 [==============================] - 14s 40ms/step - loss: 0.2437 - dice_coef: 0.7116 - val_loss: 1.3228 - val_dice_coef: 0.5682
Epoch 5/40
345/345 [==============================] - 14s 40ms/step - loss: 0.2315 - dice_coef: 0.7240 - val_loss: 1.4613 - val_dice_coef: 0.5613
Epoch 6/40
345/345 [==============================] - 14s 40ms/step - loss: 0.2300 - dice_coef: 0.7296 - val_loss: 1.3573 - val_dice_coef: 0.5685
Epoch 7/40
345/345 [==============================] - 14s 40ms/step - loss: 0.2191 - dice_coef: 0.7433 - val_loss: 1.4934 - 

## my UNET 4 (Больше слоев, стандартное окно, увеличенный эмбеддинг)

## Подготовка

In [ ]:
xLen = 256 # Длина окна
step = 15 # Шаг 
embeddingSize = 600 # Количество измерений для векторного пространства

In [ ]:
# Генерируем наборы с заданными параметрами окна
xTrain = getSetFromIndexes(decoded_text, xLen, step) # Последовательность из xLen слов
yTrain = getSetFromIndexes(yData, xLen, step) # Последовательность из xLen-тегов

In [ ]:
modelGENSIM = word2vec.Word2Vec(xTrain, size = embeddingSize, window = 10, min_count = 1, workers = 10, iter = 10)

In [ ]:
xTrainGENSIM, yTrainGENSIM = getSets(modelGENSIM, xTrain, yTrain)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


## Создание модели

In [ ]:
#Функция для создания сети UNET 
def create_unet2(k = 1, num_classes = 6, input_shape= (30, 300)):

      img_input = Input(input_shape) 

    # Block 1
    x = Conv1D(64 * k , 3, padding='same')(img_input) 
    x = Conv1D(64 * k , 3, padding='same')(x) 
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(64 * k , 3, padding='same')(x)  
    x = Conv1D(64 * k , 3, padding='same')(x)  
    x = BatchNormalization()(x)     
    block_1_out = Activation('relu')(x) 

    x = MaxPooling1D()(block_1_out)

    # Block 2
    x = Conv1D(128 * k , 3, padding='same')(x)
    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)  

    x = Conv1D(128 * k , 3, padding='same')(x)
    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_2_out = Activation('relu')(x)

    x = MaxPooling1D()(block_2_out)

    # Block 3
    x = Conv1D(256 * k , 3, padding='same')(x)
    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)               
    x = Activation('relu')(x)                     

    x = Conv1D(256 * k , 3, padding='same')(x)
    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(256 * k , 3, padding='same')(x)
    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_3_out = Activation('relu')(x)

    # x = block_3_out
    x = MaxPooling1D()(block_3_out)

    # Block 4
    x = Conv1D(512 * k , 3, padding='same')(x)
    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(512 * k , 3, padding='same')(x)
    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)      
    block_4_out = Activation('relu')(x)
    x = block_4_out 

    # UP 2
    x = Conv1DTranspose(256, kernel_size=2, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 

    x = concatenate([x, block_3_out]) 
    x = Conv1D(256 * k , 3, padding='same')(x) 
    x = Conv1D(256 * k , 3, padding='same')(x) 
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(256 * k , 3, padding='same')(x)
    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # UP 3
    x = Conv1DTranspose(128, kernel_size=2, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 

    x = concatenate([x, block_2_out])
    x = Conv1D(128 * k , 3, padding='same')(x)
    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(128 * k , 3, padding='same')(x)
    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # UP 4
    x = Conv1DTranspose(64, kernel_size=2, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = concatenate([x, block_1_out])
    x = Conv1D(64 * k , 3, padding='same')(x)
    x = Conv1D(64 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(64 * k , 3, padding='same')(x)
    x = Conv1D(64 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(num_classes, 3, activation='sigmoid', padding='same')(x)

    model = Model(img_input, x) 

    model = Model(img_input, x)
    model.compile(optimizer=Adam(0.001), 
                  loss='categorical_crossentropy',
                  metrics=[dice_coef])
  
    return model

In [ ]:
model_b_UNET2 = create_unet2(k = 4, num_classes = 6, input_shape=(xLen, embeddingSize))
model_b_UNET2.summary()

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 600)]   0                                            
__________________________________________________________________________________________________
conv1d_32 (Conv1D)              (None, 256, 256)     461056      input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_19 (BatchNo (None, 256, 256)     1024        conv1d_32[0][0]                  
__________________________________________________________________________________________________
activation_19 (Activation)      (None, 256, 256)     0           batch_normalization_19[0][0]     
_______________________________________________________________________________________

## Обучение

In [ ]:
history = model_b_UNET2.fit(xTrainGENSIM, yTrainGENSIM, epochs=60, batch_size=64, validation_split = 0.2,
                            callbacks=[model_checkpoint_callback]) # Скармливаем модели изображения, разделенные по 3 классам

Epoch 1/60
345/345 [==============================] - 79s 230ms/step - loss: 0.4063 - dice_coef: 0.6023 - val_loss: 1.2461 - val_dice_coef: 0.5049
Epoch 2/60
345/345 [==============================] - 79s 230ms/step - loss: 0.2532 - dice_coef: 0.7162 - val_loss: 1.1159 - val_dice_coef: 0.5065
Epoch 3/60
345/345 [==============================] - 79s 230ms/step - loss: 0.2299 - dice_coef: 0.7438 - val_loss: 1.5333 - val_dice_coef: 0.5552
Epoch 4/60
345/345 [==============================] - 79s 230ms/step - loss: 0.2231 - dice_coef: 0.7567 - val_loss: 1.3361 - val_dice_coef: 0.5569
Epoch 5/60
345/345 [==============================] - 79s 230ms/step - loss: 0.2206 - dice_coef: 0.7596 - val_loss: 1.6971 - val_dice_coef: 0.5611
Epoch 6/60
345/345 [==============================] - 79s 230ms/step - loss: 0.2082 - dice_coef: 0.7777 - val_loss: 2.2103 - val_dice_coef: 0.5331
Epoch 7/60
345/345 [==============================] - 82s 238ms/step - loss: 0.2186 - dice_coef: 0.7667 - val_loss: 6.

In [ ]:
model_b_UNET2.load_weights('/content/drive/MyDrive/27 Сегментация теста/savings')

In [ ]:
history = model_b_UNET2.fit(xTrainGENSIM, yTrainGENSIM, epochs=60, batch_size=64, validation_split = 0.2,
                            callbacks=[model_checkpoint_callback]) # Скармливаем модели изображения, разделенные по 3 классам

Epoch 1/60
345/345 [==============================] - 79s 230ms/step - loss: 0.2101 - dice_coef: 0.7782 - val_loss: 1.4064 - val_dice_coef: 0.5693
Epoch 2/60
345/345 [==============================] - 79s 230ms/step - loss: 0.2089 - dice_coef: 0.7806 - val_loss: 1.4519 - val_dice_coef: 0.5825
Epoch 3/60
345/345 [==============================] - 79s 230ms/step - loss: 0.2037 - dice_coef: 0.7884 - val_loss: 1.5083 - val_dice_coef: 0.5650
Epoch 4/60
345/345 [==============================] - 79s 230ms/step - loss: 0.2070 - dice_coef: 0.7814 - val_loss: 1.6852 - val_dice_coef: 0.5556
Epoch 5/60
345/345 [==============================] - 79s 230ms/step - loss: 0.2038 - dice_coef: 0.7878 - val_loss: 1.8549 - val_dice_coef: 0.5685
Epoch 6/60
345/345 [==============================] - 79s 230ms/step - loss: 0.2084 - dice_coef: 0.7787 - val_loss: 1.7363 - val_dice_coef: 0.5488
Epoch 7/60
345/345 [==============================] - 79s 230ms/step - loss: 0.2035 - dice_coef: 0.7867 - val_loss: 1.

## My_UNET 4 (Embedding слой)

## Подготовка

In [ ]:
xLen = 256 # Длина окна
step = 30 # Шаг 
embeddingSize = 600 # Количество измерений для векторного пространства

In [ ]:
# формируем наборы для обучения, которые сразу подадим в модель (без WordToVec)
xTrain = getSetFromIndexes(xData, xLen, step)  #- для эмбединга формируем xTrain не из слов, а из индексов
yTrain = getSetFromIndexes(yData, xLen, step) 

In [ ]:
xTrain = np.array(xTrain)
yTrain = np.array(yTrain)

## Создание Модели

In [ ]:
#Функция для создания сети UNET 
def create_unet2(k = 1, num_classes = 6, xLen = 256 ):
        
    img_input = Input(xLen) 

    x = Embedding(len(clean_voc)+1, embeddingSize)(img_input)
    # Block 1
    x = Conv1D(64 * k , 3, padding='same')(x) 
    # x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(64 * k , 3, padding='same')(x)  
    # x = BatchNormalization()(x)     
    block_1_out = Activation('relu')(x) 

    x = MaxPooling1D()(block_1_out)

    # Block 2
    x = Conv1D(128 * k , 3, padding='same')(x)
    # x = BatchNormalization()(x)
    x = Activation('relu')(x)  

    x = Conv1D(128 * k , 3, padding='same')(x)
    # x = BatchNormalization()(x)
    block_2_out = Activation('relu')(x)

    x = MaxPooling1D()(block_2_out)

    # Block 3
    x = Conv1D(256 * k , 3, padding='same')(x)
    # x = BatchNormalization()(x)               
    x = Activation('relu')(x)                     

    x = Conv1D(256 * k , 3, padding='same')(x)
    # x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(256 * k , 3, padding='same')(x)
    # x = BatchNormalization()(x)
    block_3_out = Activation('relu')(x)

    # x = block_3_out
    x = MaxPooling1D()(block_3_out)

    # Block 4
    x = Conv1D(512 * k , 3, padding='same')(x)
    # x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(512 * k , 3, padding='same')(x)
    # x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(512 * k , 3, padding='same')(x)
    # x = BatchNormalization()(x)      
    block_4_out = Activation('relu')(x)
    x = block_4_out 

    # UP 2
    x = Conv1DTranspose(256, kernel_size=2, strides=2, padding='same')(x)
    # x = BatchNormalization()(x)
    x = Activation('relu')(x) 

    x = concatenate([x, block_3_out]) 
    x = Conv1D(256 * k , 3, padding='same')(x) 
    # x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(256 * k , 3, padding='same')(x)
    # x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # UP 3
    x = Conv1DTranspose(128, kernel_size=2, strides=2, padding='same')(x)
    # x = BatchNormalization()(x)
    x = Activation('relu')(x) 

    x = concatenate([x, block_2_out])
    x = Conv1D(128 * k , 3, padding='same')(x)
    # x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(128 * k , 3, padding='same')(x)
    # x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # UP 4
    x = Conv1DTranspose(64, kernel_size=2, strides=2, padding='same')(x)
    # x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = concatenate([x, block_1_out])
    x = Conv1D(64 * k , 3, padding='same')(x)
    # x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(64 * k , 3, padding='same')(x)
    # x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(num_classes, 3, activation='sigmoid', padding='same')(x)

    model = Model(img_input, x) 

    model = Model(img_input, x)
    model.compile(optimizer=Adam(0.001), 
                  loss='categorical_crossentropy',
                  metrics=[dice_coef])
  
    return model

In [ ]:
model_b_UNET2 = create_unet2(k=4, xLen=256)
model_b_UNET2.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 256, 600)     3477000     input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 256, 256)     461056      embedding[0][0]                  
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 256)     0           conv1d[0][0]                     
_______________________________________________________________________________________

## ОБучение

In [ ]:
history = model_b_UNET2.fit(xTrain, yTrain, epochs=60, batch_size=64, validation_split = 0.2,
                            callbacks=[model_checkpoint_callback]) # Скармливаем модели изображения, разделенные по 3 классам

Epoch 1/60
173/173 [==============================] - 42s 245ms/step - loss: 0.6936 - dice_coef: 0.4545 - val_loss: 1.0651 - val_dice_coef: 0.4758
Epoch 2/60
173/173 [==============================] - 39s 226ms/step - loss: 0.4005 - dice_coef: 0.6000 - val_loss: 1.4435 - val_dice_coef: 0.5168
Epoch 3/60
173/173 [==============================] - 39s 225ms/step - loss: 0.2881 - dice_coef: 0.6778 - val_loss: 1.6057 - val_dice_coef: 0.5285
Epoch 4/60
173/173 [==============================] - 39s 226ms/step - loss: 0.2431 - dice_coef: 0.7162 - val_loss: 2.0711 - val_dice_coef: 0.5353
Epoch 5/60
173/173 [==============================] - 39s 226ms/step - loss: 0.2251 - dice_coef: 0.7321 - val_loss: 2.2050 - val_dice_coef: 0.5413
Epoch 6/60
173/173 [==============================] - 39s 225ms/step - loss: 0.2184 - dice_coef: 0.7427 - val_loss: 2.3986 - val_dice_coef: 0.5415
Epoch 7/60
173/173 [==============================] - 39s 226ms/step - loss: 0.2117 - dice_coef: 0.7503 - val_loss: 2.

KeyboardInterrupt: ignored

In [ ]:
model_b_UNET2 = create_unet2(xLen=128)
model_b_UNET2.summary()

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 128, 300)     1738500     input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_17 (Conv1D)              (None, 128, 64)      57664       embedding_1[0][0]                
__________________________________________________________________________________________________
activation_19 (Activation)      (None, 128, 64)      0           conv1d_17[0][0]                  
_______________________________________________________________________________________

In [ ]:
history = model_b_UNET2.fit(xTrain, yTrain, epochs=60, batch_size=64, validation_split = 0.2,
                            callbacks=[model_checkpoint_callback]) # Скармливаем модели изображения, разделенные по 3 классам

Epoch 1/60
345/345 [==============================] - 12s 34ms/step - loss: 0.6784 - dice_coef: 0.4594 - val_loss: 0.9264 - val_dice_coef: 0.4673
Epoch 2/60
345/345 [==============================] - 11s 32ms/step - loss: 0.3915 - dice_coef: 0.6026 - val_loss: 1.1142 - val_dice_coef: 0.5190
Epoch 3/60
345/345 [==============================] - 11s 32ms/step - loss: 0.2974 - dice_coef: 0.6620 - val_loss: 1.3631 - val_dice_coef: 0.5305
Epoch 4/60
345/345 [==============================] - 11s 33ms/step - loss: 0.2647 - dice_coef: 0.6847 - val_loss: 1.6332 - val_dice_coef: 0.5296
Epoch 5/60
345/345 [==============================] - 11s 32ms/step - loss: 0.2491 - dice_coef: 0.6970 - val_loss: 1.5837 - val_dice_coef: 0.5194
Epoch 6/60
345/345 [==============================] - 11s 32ms/step - loss: 0.2407 - dice_coef: 0.7057 - val_loss: 1.7274 - val_dice_coef: 0.5323
Epoch 7/60
345/345 [==============================] - 11s 32ms/step - loss: 0.2312 - dice_coef: 0.7157 - val_loss: 1.8516 - 